In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

csv_file = '/content/dataset_part1.csv'
dataset = pd.read_csv(csv_file)

In [ ]:
dataset

,Query,Response
0,What is the vision and mission of DRDO as stat...,The vision of the Defence Research & Developme...
1,What is the purpose of the Procurement Manual ...,The Procurement Manual 2020 aims to establish ...
2,What is the scope of the Procurement Manual i...,The scope of the Procurement Manual encompasse...
3,What is the effective date of the Procurement ...,The Procurement Manual 2020 came into effect f...
4,What are the exclusions from the Procurement M...,The Procurement Manual is not applicable to th...
5,What are the provisions regarding the use of s...,The manual mandates the use of standard templa...
6,How are disagreements with the Financial Advis...,If there is a disagreement between the Compete...
7,What is the procedure for deviations from the ...,Deviations from the standard procurement proce...
8,What are the banking instruments mentioned in ...,The manual outlines the use of banking instrum...
9,What is the Pre-Contract Integrity Pact in con...,The Pre-Contract Integrity Pact is a tool used...


In [ ]:
import pandas as pd


# Define the alpaca_prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

# Load your dataset from the CSV file
csv_file = '/content/dataset_part1.csv'
dataset = pd.read_csv(csv_file)

# Extract the 'Query' and 'Answer' columns
instructions = dataset["Query"].tolist()
outputs = dataset["Response"].tolist()


EOS_TOKEN = tokenizer.eos_token

# Formatting function to create the "text" column
def formatting_prompts_func(instructions, outputs):
    texts = []
    for instruction, output in zip(instructions, outputs):
        # Create the formatted text and add EOS_TOKEN
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return texts

# Generate the formatted text column
dataset['text'] = formatting_prompts_func(instructions, outputs)

In [ ]:
dataset

,Query,Response,text
0,What is the vision and mission of DRDO as stat...,The vision of the Defence Research & Developme...,"Below is an instruction that describes a task,..."
1,What is the purpose of the Procurement Manual ...,The Procurement Manual 2020 aims to establish ...,"Below is an instruction that describes a task,..."
2,What is the scope of the Procurement Manual i...,The scope of the Procurement Manual encompasse...,"Below is an instruction that describes a task,..."
3,What is the effective date of the Procurement ...,The Procurement Manual 2020 came into effect f...,"Below is an instruction that describes a task,..."
4,What are the exclusions from the Procurement M...,The Procurement Manual is not applicable to th...,"Below is an instruction that describes a task,..."
5,What are the provisions regarding the use of s...,The manual mandates the use of standard templa...,"Below is an instruction that describes a task,..."
6,How are disagreements with the Financial Advis...,If there is a disagreement between the Compete...,"Below is an instruction that describes a task,..."
7,What is the procedure for deviations from the ...,Deviations from the standard procurement proce...,"Below is an instruction that describes a task,..."
8,What are the banking instruments mentioned in ...,The manual outlines the use of banking instrum...,"Below is an instruction that describes a task,..."
9,What is the Pre-Contract Integrity Pact in con...,The Pre-Contract Integrity Pact is a tool used...,"Below is an instruction that describes a task,..."


In [ ]:
for i in range(3):
  print(i ,"th data")
  print(dataset['text'][i])
  print("\n")

0 th data
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What is the vision and mission of DRDO as stated in the document  in context of procurement chapter 1?

### Response:
The vision of the Defence Research & Development Organisation (DRDO) is to empower India with cutting-edge defense technologies. The mission is to achieve self-reliance in critical technologies while equipping the armed forces with state-of-the-art equipment, weapon systems, and platforms. DRDO aims to develop these capabilities through partnerships with industries, academic institutions, and other research organizations. Established in 1958 with only ten laboratories, DRDO has grown to become a core defense research organization in India, developing strategic and tactical military hardware that has gained both national and global recognition.<|end_of_text|>


1 th data
Below is an instr

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)

In [ ]:
dataset

,Query,Response,text
0,What is the vision and mission of DRDO as stat...,The vision of the Defence Research & Developme...,"Below is an instruction that describes a task,..."
1,What is the purpose of the Procurement Manual ...,The Procurement Manual 2020 aims to establish ...,"Below is an instruction that describes a task,..."
2,What is the scope of the Procurement Manual i...,The scope of the Procurement Manual encompasse...,"Below is an instruction that describes a task,..."
3,What is the effective date of the Procurement ...,The Procurement Manual 2020 came into effect f...,"Below is an instruction that describes a task,..."
4,What are the exclusions from the Procurement M...,The Procurement Manual is not applicable to th...,"Below is an instruction that describes a task,..."
5,What are the provisions regarding the use of s...,The manual mandates the use of standard templa...,"Below is an instruction that describes a task,..."
6,How are disagreements with the Financial Advis...,If there is a disagreement between the Compete...,"Below is an instruction that describes a task,..."
7,What is the procedure for deviations from the ...,Deviations from the standard procurement proce...,"Below is an instruction that describes a task,..."
8,What are the banking instruments mentioned in ...,The manual outlines the use of banking instrum...,"Below is an instruction that describes a task,..."
9,What is the Pre-Contract Integrity Pact in con...,The Pre-Contract Integrity Pact is a tool used...,"Below is an instruction that describes a task,..."


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 50 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.691500
2,2.709500
3,2.594300
4,2.655400
5,2.504000
6,2.207900
7,2.010700
8,1.921800
9,1.755700
10,1.482200


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is the scope of the Procurement Manual  in context of procurement chapter 1? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# The scope of the Procurement Manual encompasses all kinds of goods, stores, services, and development contracts for DRDO.
#  It applies to the acquisition of various services, including outsourcing, transportation, insurance, and consultancy, among others.
#   The manual also covers the procurement of job contracts related to packaging, software development, and technical assessment.
#    However, for civil works beyond the installation of equipment, consultation with DRDOs Civil Works and Estates department is required.
# The manual's procedures ensure that procurement activities align with the needs of DRDO while following governmental guidelines.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is the scope of the Procurement Manual  in context of procurement chapter 1? 

### Response:
 The scope of the Procurement Manual encompasses all kinds of goods, stores, services, and development contracts for DRDO. It applies to the acquisition of various items, including civil, electrical, and professional services. The manual also covers the procurement of job contracts for outsourced development work. The procedures outlined are intended to ensure efficiency, economy, transparency, and fair dealing in all procurement activities. The manual is designed to comply with the guidelines issued by the Central Government and the Ministry of Defence (R&D).<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What is the vision and mission of DRDO as stated in the document? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# The vision of the Defence Research & Development Organisation (DRDO) is to empower India with cutting-edge defense technologies.
# The mission is to achieve self-reliance in critical technologies while equipping the armed forces with state-of-the-art equipment,
#  weapon systems, and platforms. DRDO aims to develop these capabilities through partnerships with industries, academic institutions,
#   and other research organizations. Established in 1958 with only ten laboratories, DRDO has grown to become a core defense research
#   organization in India,
# developing strategic and tactical military hardware that has gained both national and global recognition.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What is the vision and mission of DRDO as stated in the document? 

### Response:
 The vision of the Defence Research & Development Organisation (DRDO) is to empower India with cutting-edge defense technologies. The mission is to achieve self-reliance in critical technologies while equipping the armed forces with state-of-the-art equipment, weaponry, and platforms. DRDO aims to develop these capabilities through partnerships with industries, academic institutions, and other research organizations while focusing on innovation and technology development. Established in 1958 with only ten laboratories, DRDO has grown to become a core defense research organization, contributing significantly to Indias defense capabilities.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What are the different categories of indigenous firms for registration? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Indigenous firms are categorized into several types for registration based on their capabilities: 1) Manufacturers/Distributors - for firms that
#  manufacture or are authorized stockists of Commercially-Off-The-Shelf (COTS) items; 2) Service Providers - for firms offering
#   professional services such as maintenance and outsourced jobs; 3) Fabrication/Production Agencies - for firms that handle production
#    without contributing intellectual property; 4) Development and Production Agencies - for firms capable of development
#     and manufacturing without design infrastructure; 5) Design, Development, and
# Production Agencies - for firms capable of design, research, and manufacturing. Other firms may be registered under the Others category.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What are the different categories of indigenous firms for registration? 

### Response:
 Several categories of indigenous firms exist for registration under the document: Indigenous Firms for Procurement of Goods. These include: 1) Manufacturers/Distributors/Wholesalers - for firms that manufacture or are authorized stockists or distributors of the goods; 2) Service Providers - for firms that offer services related to the procurement; 3) CFAAT-Registered Firms - for firms registered with the Central Fertilizer Advisory Agency and Testing (CFAAT); 4) Government Designated Firms - for firms designated by government departments or organizations as reliable sources; 5) Others - for firms that do not fall


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What are the exclusions from the Procurement Manual? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# The Procurement Manual is not applicable to the creation of civil infrastructure. Additionally, procurements made under long-term
# government agreements or inter-government agreements between India and other countries will follow the terms outlined in those
#  agreements, overriding the provisions of the manual. These procurements are treated similarly to PAC procurements, ensuring
#   that the terms and conditions, quotations, and other relevant aspects are governed by the government agreements rather than
#   the manual.
#  This allows for flexibility in cases where government-to-government agreements are in place.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What are the exclusions from the Procurement Manual? 

### Response:
 The Procurement Manual is not applicable to the creation of civil infrastructure. Additionally, procurement of stores for outlays made under the heads of Equipment and Plant Maintenance, Minor Purchase, and Outfits and Accidents does not fall under the scope of this manual. These procurements are handled through existing channels within the respective departments. The manual also excludes the procurement of stores for outlays made under the heads of Civil Works and Services, where specific guidelines are provided by the respective departments.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " How does the Local Purchase Committee (LPC) conduct minor purchases? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# The Local Purchase Committee (LPC) is responsible for handling minor purchases under Rule 155 of GFR-2017. The LPC consists of
#  at least three members who conduct a market survey to ascertain the quality, specifications, and price of goods or services.
#  The committee ensures that the vendor selected is reliable and competent. LPC members are required to certify that the goods
#   meet the requisite specifications and that the price is reasonable compared to prevailing market rates.
#   CFA approval is required before
# the LPC can proceed with the purchase, and all relevant forms and documentation are maintained for transparency and audit purposes.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 How does the Local Purchase Committee (LPC) conduct minor purchases? 

### Response:
 The Local Purchase Committee (LPC) conducts minor purchases through verbal inquiries or by circulating a petty purchase requisition among reliable sources. The LPC ensures that the vendor is reliable and that the price is reasonable and competitive. The LPC must follow strict guidelines to prevent misuse of powers. No vendor is allowed to participate in multiple LPCs, and the purchases must be made within the prescribed petty purchase rules. The LPC secretary maintains discipline and order during the procurement process.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "under which Rule Local Purchase Committee (LPC) conduct minor purchases?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# The Local Purchase Committee (LPC) is responsible for handling minor purchases under Rule 155 of GFR-2017.
#  The LPC consists of at least three members who conduct a market survey to ascertain the quality, specifications, and price
#  of goods or services. The committee ensures that the vendor selected is reliable and competent. LPC members are required to certify
#   that the goods meet the requisite specifications and that the price is reasonable compared to prevailing market rates. CFA approval
#   is required before
#  the LPC can proceed with the purchase, and all relevant forms and documentation are maintained for transparency and audit purposes.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
under which Rule Local Purchase Committee (LPC) conduct minor purchases?

### Response:
The Local Purchase Committee (LPC) is authorized to conduct minor purchases under Rule 155 of GFR-2017 for items or services required for operational or developmental needs. The LPC must consist of at least two members, including a chairman, who ensures that the procurement complies with the rules and regulations. The committee is responsible for assessing the need for the purchase, determining the specifications, and ensuring that the vendor selected is reliable and competent. LPC members are required to certify that the purchase is necessary and that the price is reasonable compared to prevailing market rates.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " What are the rules for the procurement of goods for trials, exhibitions, or launch campaigns? ", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

# Rule 21 of GFR 2017 emphasizes that every officer incurring or authorizing expenditure from
#  public money should be guided by high standards of financial propriety and enforce financial order and strict economy.What is the vision and mission of DRDO as stated in the document?

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 What are the rules for the procurement of goods for trials, exhibitions, or launch campaigns? 

### Response:
 The procurement of goods for trials, exhibitions, or launch campaigns must follow specific rules. These procurements are typically made under Rule 154 of GFR-2017, which allows for the direct purchase of goods without competitive bidding. The goods must be necessary for the trial, exhibition, or launch campaign, and the procurement must be limited to the quantities needed for the specific event. The trial, exhibition, or launch campaign must be approved by the Competent Financial Authority (CFA) or the Competent Procurement Authority (CPA), depending on the value of the procurement. The procurement process must be completed within the specified time frame,


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " under how many days Consolidated Receipt Voucher (CRV) must be prepared in procurement ?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

#  Consolidated Receipt Voucher (CRV) must be prepared within 15 days to account for the purchased goods.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 under how many days Consolidated Receipt Voucher (CRV) must be prepared in procurement?

### Response:
 According to Rule 155 of GFR-2017, a Consolidated Receipt Voucher (CRV) must be prepared within 15 days of the receipt of goods or services for all procurements, including the procurement of goods from the Government e-Marketplace (GeM). The CRV is used to consolidate all payments made towards the procurement, including advances, partial payments, and the balance amount. This process ensures that all financial obligations are settled, and no further payments are made beyond the total value specified in the Purchase Request. The CRV is a critical step in closing out the procurement transaction.<|end_of_text|>


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        " under how many days Consolidated Receipt Voucher (CRV) must be prepared?", # instruction

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

#  Consolidated Receipt Voucher (CRV) must be prepared within 15 days to account for the purchased goods.

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
 under how many days Consolidated Receipt Voucher (CRV) must be prepared?

### Response:
 According to the document, a Consolidated Receipt Voucher (CRV) must be prepared within 45 days of the receipt of money or goods, or the completion of a contract, whichever occurs later. This ensures that all money received is accounted for and recorded accurately. The CRV is a critical step in maintaining financial discipline and transparency. Failure to submit a CRV within the stipulated time may result in the initiation of disciplinary action against the concerned officer or lab.<|end_of_text|>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save_pretrained("/content/drive/MyDrive/Images/lora_model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/Images/lora_model")

('/content/drive/MyDrive/Images/lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/Images/lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/Images/lora_model/tokenizer.json')

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "/content/drive/MyDrive/Images/lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!


==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Unsloth 2024.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""



inputs = tokenizer(
[
    alpaca_prompt.format(
        "under which Rule Local Purchase Committee (LPC) conduct minor purchases?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
under which Rule Local Purchase Committee (LPC) conduct minor purchases?

### Input:


### Response:
The Local Purchase Committee (LPC) is authorized to conduct minor purchases under Rule 155 of GFR-2017 for items such as stationery, repair services, and outsourced jobs. The LPC is composed of at least three members who conduct a market survey to ascertain the price of the goods or services required. The LPC must ensure that the purchase does not exceed the prescribed petty purchase limits. The committee's recommendations are forwarded to the Head of the Division for approval, and the purchase is carried out under strict guidelines to ensure transparency and fairness.<|end_of_text|>


In [ ]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        "under how many days Consolidated Receipt Voucher (CRV) must be prepared in procurement ?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
under how many days Consolidated Receipt Voucher (CRV) must be prepared in procurement?

### Input:


### Response:
According to Rule 155 of GFR-2017, a Consolidated Receipt Voucher (CRV) must be prepared within 15 days of the receipt of goods, money, or securities. This ensures that all relevant details are captured and processed promptly. Failure to submit a CRV within the stipulated time may result in delays in the release of payments or other financial transactions. The CRV is essential for maintaining accurate financial records and closing out procurement cases.<|end_of_text|>


In [ ]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What role does the Competent Financial Authority (CFA) play in procurement in context of procurement chapter 4?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What role does the Competent Financial Authority (CFA) play in procurement in context of procurement chapter 4?

### Input:


### Response:
The Competent Financial Authority (CFA) plays a crucial role in the procurement process as its approval is required at various stages. Once the Demand Splitting Committee (DSC) approves the demand, it is forwarded to the CFA for approval based on the total cost and mode of bidding. The CFA ensures that the demand complies with financial regulations and that the procurement is necessary and justified. The CFA's approval is also needed before releasing the necessary funds for the procurement. This step is essential to prevent expenditures that are not essential or beyond what is necessary.<|end_of_text|>


In [ ]:

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Drug and medicine purchases handling for laboratory dispensaries  in context of procurement ?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
How are drug and medicine purchases handled for laboratory dispensaries  in context of procurement chapter 5?

### Input:


### Response:
Laboratories and establishments with dispensaries can procure medicines through regular supply channels or directly from the Armed Forces Medical Store Depots (AFMSD). Quarterly demands are submitted to ensure continuous supply. Medicines are typically stocked based on the last quarter's consumption, and efforts are made to prevent stock outs. In cases where immediate supply is required, the Local Purchase Committee (LPC) can be convened for on-site purchases. The Medicinal Stores Department (MSD) at DRDO Headquarters maintains records of stock levels and initiates supply actions as needed.<|end_of_text|>
